In [ ]:
# Import des bibliothèques nécessaires

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split


# import logging  # À voir si on l'ajoute pour mieux structurer les messages dans la console

In [ ]:
# Chargement du fichier dataset

dataset = "data/Teacher/NSL_KDD.csv"

In [ ]:
# Question 4 - Renommer les colonnes du dataset

## Liste des nouveaux noms de colonnes

col_names = ["duration","protocol_type","service","flag","src_bytes",
"dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
"logged_in","num_compromised","root_shell","su_attempted","num_root",
"num_file_creations","num_shells","num_access_files","num_outbound_cmds",
"is_host_login","is_guest_login","count","srv_count","serror_rate",
"srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
"diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
"dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
"dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
"dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

## Modification des noms de colonnes
dataset = pd.read_csv(dataset, header=None, names=col_names)

## Affichage du dataset d'entraînement avec les nouveaux noms de colonnes
print("Dataset avec les nouveaux noms de colonnes :")
display(dataset)

In [ ]:
# Question 2 - Visualiser les valeurs de la colonne 'label' et tracer leur répartition

colonne_label = dataset['label'] # Sélection de la colonne 'label' du DataFrame

## Suppression des valeurs 'normal'
colonne_label = colonne_label[colonne_label != "normal"]

ax = colonne_label.value_counts().plot(
    kind='bar',
    title='Répartition des types d\'attaques',
    figsize=(10, 6),
)

ax.set_xlabel('Groupe d\'attaque')
ax.set_ylabel("Nombre d'occurrences")
plt.xticks(rotation=45, ha='right')

## Ajouter les valeurs au-dessus des barres
for p in ax.patches:
    ax.text(
        p.get_x() + p.get_width() / 2,   # position horizontale (au centre de la barre)
        p.get_height() + 0.5,            # position verticale (légèrement au-dessus)
        int(p.get_height()),             # valeur affichée
        ha='center',                     # centrer horizontalement
        va='bottom'                      # aligner en bas
    )

plt.tight_layout()
plt.show()

In [ ]:
# Question 3 - Analyse des valeurs manquantes avec une heatmap

# Calcul des valeurs manquantes dans le dataset d'entraînement
missing_train = dataset.isnull().sum()

# Créer et afficher une carte de chaleur pour visualiser les valeurs manquantes
plt.figure(figsize=(12, 6))
sns.heatmap(
    dataset.isnull(),
    cbar=True,
    cmap='viridis',
    cbar_kws={'label': 'Valeur manquante (True/False)'}
)
plt.title("Valeurs manquantes dans le dataset")
plt.xlabel('Colonnes')
plt.ylabel('Index des lignes')
plt.show()


In [ ]:
# Question 5 - Vérifier le nombre de lignes dupliquées dans le dataset d'entraînement

print("Nombre total de lignes dupliquées dans le dataset d'entraînement :", dataset.duplicated().sum())

In [ ]:
# Question 6 - Suppression des doublons

print("Avant suppression, nombre de lignes dans le dataset d'entraînement :", len(dataset))
dataset.drop_duplicates(inplace=True) # Supprimer les doublons
print("Après suppression, nombre de lignes dans le dataset d'entraînement :", len(dataset))

In [ ]:
# Question 10 - Reprendre le dataset d'origine (avant autres traitements) pour la question 10

train_data_q10 = dataset.copy() # Faire une copie pour ne pas modifier l'original

In [ ]:
# Question 7 - Encodage des variables catégorielles

## Identifier les colonnes catégorielles
cat_columns = dataset.select_dtypes(include=['object']).columns.tolist()
print("Colonnes catégorielles trouvées :", cat_columns)

## Afficher les colonnes des valeurs catégorielles
for col in cat_columns:
    print(f"\nColonne : {col}")
    print(dataset[col].value_counts())  # Affiche les valeurs uniques avec leur nombre

## Séparer les colonnes selon leur nombre de valeurs uniques
cat_columns_few_values = []  # Colonnes avec moins de 5 valeurs uniques
cat_columns_many_values = []  # Colonnes avec 5 valeurs uniques ou plus

for col in cat_columns:
    unique_count = dataset[col].nunique()
    if unique_count < 5:
        cat_columns_few_values.append(col)
    else:
        cat_columns_many_values.append(col)

print(f"\nColonnes catégorielles avec moins de 5 valeurs uniques : {cat_columns_few_values}")
print(f"Colonnes catégorielles avec 5 valeurs uniques ou plus : {cat_columns_many_values}")

In [ ]:
# Question 7 (suite) - Encodage des variables catégorielles

print(f"Nombre de colonnes avant encodage : {len(dataset.columns)}")


## Encoder chaque colonne catégorielle avec LabelEncoder
for col in cat_columns_many_values:
    dataset[col] = LabelEncoder().fit_transform(dataset[col])



## Encoder chaque colonne catégorielle avec OneHotEncoder
enc = OneHotEncoder(sparse_output=False, dtype='int')  # dtype='int' pour avoir des entiers au lieu de float

### Traiter chaque colonne catégorielle une par une avec une boucle
for col in cat_columns_few_values:
    print(f"Traitement de la colonne : {col}")
    
    
    # Préparer les données pour OneHotEncoder (reshape en 2D)
    X = dataset[col].values.reshape(-1, 1)
    
    # Encoder la colonne
    encoded_data = enc.fit_transform(X)
    
    # Créer les noms des nouvelles colonnes
    feature_names = enc.get_feature_names_out([col])
    
    # Créer un DataFrame avec les données encodées
    encoded_df = pd.DataFrame(encoded_data, columns=feature_names, index=dataset.index)
    
    # Ajouter les nouvelles colonnes au dataset
    dataset = pd.concat([dataset, encoded_df], axis=1)
    
    # Supprimer l'ancienne colonne du dataset
    dataset = dataset.drop(columns=[col])
    
    print(f"Colonnes ajoutées: {list(feature_names)}")
    print(f"Colonne {col} supprimée\n")

print(f"Nombre de colonnes après encodage : {len(dataset.columns)}")

## Sauvegarder les noms des colonnes après encodage
col_names_encoded = list(dataset.columns)
print(f"Noms des colonnes sauvegardés : {col_names_encoded}")

display(dataset)

In [ ]:

# Question 8 - Normalisation / Mise à l'échelle

## Copy du dataset pour éviter de modifier l'original
dataset_StandardScaler = dataset.copy()
dataset_MinMaxScaler = dataset.copy()
dataset_RobustScaler = dataset.copy()

## Afficher les valeurs aberrantes avant normalisation
sns.boxplot(data=dataset)
plt.title("Valeurs aberrantes avant normalisation")
plt.show()


## Appliquer RobustScaler
dataset_RobustScaler = RobustScaler().fit_transform(dataset_RobustScaler)

sns.boxplot(data=dataset_RobustScaler)
plt.title("Valeurs aberrantes après normalisation RobustScaler")
plt.show()

## Appliquer MinMaxScaler
dataset_MinMaxScaler = MinMaxScaler().fit_transform(dataset_MinMaxScaler)

sns.boxplot(data=dataset_MinMaxScaler)
plt.title("Valeurs aberrantes après normalisation MinMaxScaler")
plt.show()

## Appliquer StandardScaler
dataset_StandardScaler = StandardScaler().fit_transform(dataset_StandardScaler)
sns.boxplot(data=dataset_StandardScaler)
plt.title("Valeurs aberrantes après normalisation StandardScaler")
plt.show()

In [ ]:
# Question 8 (suite) - Normalisation / Mise à l'échelle

## Utilisation de la fonction StandardScaler
dataset = StandardScaler().fit_transform(dataset)

# Reconvertir l'array NumPy en DataFrame pandas (nécessaire après StandardScaler)
dataset = pd.DataFrame(dataset)

## Réassigner les noms de colonnes au DataFrame
dataset.columns = col_names_encoded

display(dataset)

In [ ]:
# Question 9 - Découpage en ensembles (train / test)

# Séparation des features (X) et du target (y)
X = dataset.drop('label', axis=1)  # Toutes les colonnes sauf 'label'
y = dataset['label']  # Seulement la colonne 'label'

# Découpage train/test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Affichage du nombre de lignes dans chaque jeu
print("Nombre de lignes dans le jeu d'entraînement final :", len(X_train))
print("Nombre de lignes dans le jeu de test final :", len(X_test))

## Afficher les tailles et la répartition des classes
print(f"\nTaille des ensembles:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

## Sauvegarde des fichiers en format CSV (Optionnel)
out_dir = 'data/Dataset_pre_traiter/first_part'

# Sauvegarder chaque objet (features et target) dans des fichiers séparés
X_train.to_csv(f'{out_dir}/X_train.csv', index=False)
y_train.to_csv(f'{out_dir}/y_train.csv', index=False)
X_test.to_csv(f'{out_dir}/X_test.csv', index=False)
y_test.to_csv(f'{out_dir}/y_test.csv', index=False)

print(f'Données sauvegardées dans le dossier : {out_dir}')

In [ ]:
# Question 10 (suite) - Regroupement des attaques en grandes familles (DOS, Probe, R2L, U2R)

## Listes fournies stockées dans des variables
dos = [
    "back","land","neptune","pod","smurf","teardrop","mailbomb",
    "processtable","udpstorm","apache2","worm"
]

probe = [
    "satan","ipsweep","nmap","portsweep","mscan","saint"
]

r2l = [
    "guess_passwd","ftp_write","imap","phf","multihop","warezmaster",
    "xlock","xsnoop","snmpguess","snmpgetattack","httptunnel","sendmail","named", "warezclient"
]

u2r = [
    "buffer_overflow","loadmodule","rootkit","perl","sqlattack","xterm","ps"
]

normal = ["normal"]

## Construire un dictionnaire de correspondance (label -> grand groupe)
mapping = {}
for name in dos:
    mapping[name] = 'DOS'
for name in probe:
    mapping[name] = 'Probe'
for name in r2l:
    mapping[name] = 'R2L'
for name in u2r:
    mapping[name] = 'U2R'
for name in normal:
    mapping[name] = 'Normal'

## Lecture / extraction de la colonne 'label'
labels_raw = train_data_q10['label'].astype(str)  # S'assurer que la colonne est du texte

## Mettre 'Other' pour les labels inconnus (non présents dans les listes fournies)
labels_grouped = labels_raw.apply(lambda x: mapping.get(x, 'Other')) 

## Afficher les comptes par grand groupe
print("Comptes par grand groupe :")
print(labels_grouped.value_counts())
print("\n")

## Ajouter la nouvelle colonne au jeu d'entraînement et afficher le résultat
train_data_q10['attack_family'] = labels_grouped
print("Jeu d'entraînement avec la nouvelle colonne 'attack_family' :")
display(train_data_q10)
print("\n")
print("Voir les lignes ou il y a la valeur 'Other' dans la colonne 'attack_family' :")
display(train_data_q10[train_data_q10['attack_family'] == 'Other'])
print("\n")

## Découpage par famille d'attaque en jeux distincts

dataset_DOS = train_data_q10[train_data_q10['attack_family'] == 'DOS']
dataset_Probe = train_data_q10[train_data_q10['attack_family'] == 'Probe']
dataset_R2L = train_data_q10[train_data_q10['attack_family'] == 'R2L']
dataset_U2R = train_data_q10[train_data_q10['attack_family'] == 'U2R']
dataset_Normal = train_data_q10[train_data_q10['attack_family'] == 'Normal']

## Liste de tous les jeux pour des boucles ultérieures
list_datasets = [dataset_DOS, dataset_Probe, dataset_R2L, dataset_U2R, dataset_Normal]
list_names = ['DOS', 'Probe', 'R2L', 'U2R', 'Normal']

## Vérification des jeux créés (boucle où "i" est l'index et "dataset" est la variable concrète)
for i, dataset in enumerate(list_datasets):
    print(f"Affichage du dataset {list_names[i]} : ")
    display(dataset)
    print("\n")

In [ ]:
# Question 10 (suite) - Encodage des variables catégorielles

## Pour éviter les warnings de pandas lors de la modification de copies de DataFrames
import warnings
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)


for dataset in list_datasets:

    ## Identifier les colonnes catégorielles
    cat_columns = dataset.select_dtypes(include=['object']).columns.tolist()
    print("Colonnes catégorielles trouvées :", cat_columns)

    ## Encoder chaque colonne catégorielle que j'avais repérée
    le = LabelEncoder()  # Utilisation de LabelEncoder
    for col in cat_columns:
        dataset[col] = le.fit_transform(dataset[col])

    display(dataset)
    print("\n")

In [ ]:
# Question 10 (suite) - Normalisation / Mise à l'échelle

for i, dataset in enumerate(list_datasets):
    print(f"Traitement du dataset {list_names[i]} : ")
    ## Utilisation de la fonction StandardScaler
    dataset = StandardScaler().fit_transform(dataset)

    ## Reconvertir l'array NumPy en DataFrame pandas (nécessaire après StandardScaler)
    dataset = pd.DataFrame(dataset)

    ## Liste des nouveaux noms de colonnes

    col_names_new = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label", "attack_family"]

    ## Réassigner les noms de colonnes au DataFrame
    dataset.columns = col_names_new

    display(dataset)
    print("\n")

In [ ]:
# Question 10 (suite) - Découpage en ensembles (train / test)

for i, dataset in enumerate(list_datasets):

    ## Suppression de la colonne "attack_family" car elle reste à zéro
    dataset = dataset.drop(columns=['attack_family'])

    # Séparation des features (X) et du target (y)
    X = dataset.drop('label', axis=1)  # Toutes les colonnes sauf 'label'
    y = dataset['label']  # Seulement la colonne 'label'

    # Découpage train/test (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    ## Affichage du nombre de lignes dans chaque jeu
    print(f"Dataset {list_names[i]} :")
    print("Nombre de lignes dans le jeu d'entraînement final :", len(X_train))
    print("Nombre de lignes dans le jeu de test final :", len(X_test))

    ## Afficher les tailles et la répartition des classes
    print(f"\nTaille des ensembles:")
    print(f"X_train: {X_train.shape}")
    print(f"X_test: {X_test.shape}")
    print(f"y_train: {y_train.shape}")
    print(f"y_test: {y_test.shape}")

    print(f"\nRépartition des classes dans l'ensemble d'entraînement:")
    print(y_train.value_counts())

    print(f"\nRépartition des classes dans l'ensemble de test:")
    print(y_test.value_counts())

    ## Sauvegarde des fichiers en format CSV (Optionnel)
    ### Reconstruction des DataFrames complets pour la sauvegarde
    train_data_final = pd.concat([X_train, y_train], axis=1)
    test_data_final = pd.concat([X_test, y_test], axis=1)

    ### Sauvegarde
    train_data_final.to_csv(f"data/Dataset_pre_traiter/NSL_KDD_train_final_{list_names[i]}.csv", index=False)
    test_data_final.to_csv(f"data/Dataset_pre_traiter/NSL_KDD_test_final_{list_names[i]}.csv", index=False)

    print("---------------------------------------------------")
    print("\n")
